# Unit 5 - Financial Planning


In [ ]:
# Initial imports
import os
import time
import requests
import datetime
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv("../../../resources/alpaca_keys.env")

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [ ]:
# Fetch current BTC price
current_btc_price = requests.get(btc_url).json()['data']['1']['quotes']['CAD']['price']

# Fetch current ETH price
current_eth_price = requests.get(eth_url).json()['data']['1027']['quotes']['CAD']['price']

# Compute current value of my crpto
my_btc_value = my_btc * current_btc_price
my_eth_value = my_eth * current_eth_price

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
alpaca_api_key,
alpaca_secret_key,
api_version = "v2"
)

In [ ]:
# Format current date as ISO format
current_date = pd.Timestamp("2021-04-21", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
current_price_df = api.get_barset(
tickers,
timeframe,
end = current_date,
limit = 1).df

# Preview DataFrame
current_price_df.head()

In [ ]:
# Pick AGG and SPY close prices
agg_close_price = float(current_price_df['AGG']['close'])
spy_close_price = float(current_price_df['SPY']['close'])

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price:0.2f}")
print(f"Current SPY closing price: ${spy_close_price:0.2f}")

In [ ]:
# Compute the current value of shares
my_agg_value = my_agg * agg_close_price
my_spy_value = my_spy * spy_close_price

# Print current value of share
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = 12000

# Create savings DataFrame
index = ['crypto', 'shares']
columns = ['amount']
df_savings = pd.DataFrame(index = index, 
                          columns = columns)

df_savings.loc['crypto'] = sum([my_btc_value, my_eth_value])
df_savings.loc['shares'] = sum([my_agg_value, my_spy_value])
 
# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
df_savings.plot.pie(y="amount",
                   title="Total Savings")

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_savings = float(df_savings.sum())

# Validate saving health
if total_savings > emergency_fund:
    print(f"Congratulations on passing your goal! Your savings of ${total_savings:0.2f} is \u2248 {(total_savings/emergency_fund):1.0f} times greater than your emergency fund of ${emergency_fund:0.2f}, you have enough money in your fund!")
elif total_savings == emergency_fund:
    print(f"Congratulations on reaching your goal! Your savings of ${total_savings:0.2f} is equal to your emergency fund, keep it up!")
else:
    print(f"Your savings of ${total_savings:0.2f} is ${(emergency_fund-total_savings):0.2f} short of your goal, keep saving!")


## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2016-04-20', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-04-20', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
df_stock_data = api.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date,
    limit = 1000).df.dropna()

# Get the initial date of the dataframe
last_date = list(df_stock_data.index)[0].isoformat()

# While initial data of the datafram is after start_date, run another call
while last_date >= start_date:
    print(f"Alpaca API data pulled up until {last_date}, initiating another call.")
    
    df_stock_data_again = api.get_barset(
        tickers,
        timeframe,
        start = start_date,
        end = last_date,
        limit = 1000).df.dropna()
    
    # Concatenate the df's
    df_stock_data = pd.concat([df_stock_data, df_stock_data_again], axis = "rows", join = "outer")
    last_date = str(list(df_stock_data_again.index)[0])
    time.sleep(1) 

In [ ]:
# Display sample data
df_stock_data.sort_index().head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
weights = [0.4, 0.6]
years = 30

MC_thirty_year = MCSimulation(
    portfolio_data = df_stock_data,
    weights = weights,
    num_simulation = 500,
    num_trading_days = 252 * years)

In [ ]:
# Printing the simulation input data
MC_thirty_year.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_thirty_year.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
line_plot = MC_thirty_year.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot = MC_thirty_year.plot_distribution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
stats = MC_thirty_year.summarize_cumulative_return()

# Print summary statistics
print(stats)

### Calculate the expected portfolio return at the 95% lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower = round(stats[8] * initial_investment,2)
ci_upper = round(stats[9] * initial_investment, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower = round(stats[8] * initial_investment,2)
ci_upper = round(stats[9] * initial_investment, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
weights = [0.4, 0.6]
years = 5

MC_five_year = MCSimulation(
    portfolio_data = df_stock_data,
    weights = weights,
    num_simulation = 500,
    num_trading_days = 252 * years)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_five_year.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
line_plot_five_year = MC_five_year.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot_five_year = MC_five_year.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
stats_five_year = MC_five_year.summarize_cumulative_return()

# Print summary statistics
print(stats_five_year)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_five = round(stats_five_year[8] * initial_investment,2)
ci_upper_five = round(stats_five_year[9] * initial_investment, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
weights = [0.4, 0.6]
years = 10

MC_ten_year = MCSimulation(
    portfolio_data = df_stock_data,
    weights = weights,
    num_simulation = 500,
    num_trading_days = 252 * years)

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
MC_ten_year.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
line_plot_ten_year = MC_ten_year.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot_ten_year = MC_ten_year.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
stats_ten_year = MC_ten_year.summarize_cumulative_return()

# Print summary statistics
print(stats_ten_year)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_ten = round(stats_five_year[8] * initial_investment,2)
ci_upper_ten = round(stats_five_year[9] * initial_investment, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")